<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Pure_implimentation_SWAG_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
def X_activation(x):
    return x

def X_activation_derivative(x):
    return np.ones_like(x)

def X_2_activation(x):
    return (x**2) / 8

def X_2_activation_derivative(x):
    return (x / 4)

def X_3_activation(x):
    return (x**3) / 24

def X_3_activation_derivative(x):
    return (x**2) / 8

class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
        # Define the architecture
        self.input_size = input_size
        self.hidden1_size = hidden1_size
        self.hidden2_size = hidden2_size
        self.hidden3_size = hidden3_size
        self.output_size = output_size

        # Initialize weights and biases
        self.weights = {
            'W1': np.random.randn(input_size, hidden1_size),
            'W2': np.random.randn(input_size, hidden2_size),
            'W3': np.random.randn(input_size, hidden3_size),
            'W4': np.random.randn(hidden1_size + hidden2_size + hidden3_size, output_size)
        }

        self.biases = {
            'b1': np.random.randn(1, hidden1_size),
            'b2': np.random.randn(1, hidden2_size),
            'b3': np.random.randn(1, hidden3_size),
            'b4': np.random.randn(1, output_size)
        }

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def X_activation(self, x):
        return X_activation(x)

    def X_activation_derivative(self, x):
        return X_activation_derivative(x)

    def X_2_activation(self, x):
        return X_2_activation(x)

    def X_2_activation_derivative(self, x):
        return X_2_activation_derivative(x)

    def X_3_activation(self, x):
        return X_3_activation(x)

    def X_3_activation_derivative(self, x):
        return X_3_activation_derivative(x)

    def feedforward(self, X):
        # Layer 1 (X Activation)
        self.z1 = np.dot(X, self.weights['W1']) + self.biases['b1']
        self.a1 = self.X_activation(self.z1)  # Use X activation for layer

        # Layer 2
        self.z2 = np.dot(X, self.weights['W2']) + self.biases['b2']
        self.a2 = self.X_2_activation(self.z2)

        # Layer 3
        self.z3 = np.dot(X, self.weights['W3']) + self.biases['b3']
        self.a3 = self.X_3_activation(self.z3)

        # Concatenate the outputs of layers 1, 2, and 3
        self.concatenated_output = np.concatenate((self.a1, self.a2, self.a3), axis=1)

        # Layer 4 (Output Layer)
        self.z4 = np.dot(self.concatenated_output, self.weights['W4']) + self.biases['b4']
        self.output = self.X_activation(self.z4)

        return self.output

    def backpropagation(self, X, y, learning_rate):
        # Backpropagation

        # Layer 4 (Output Layer)
        delta4 = 2 * (self.output - y) * self.X_activation_derivative(self.output)
        dW4 = np.dot(self.concatenated_output.T, delta4)
        db4 = np.sum(delta4, axis=0, keepdims=True)

        # Split the delta for the concatenation in layer 4
        delta4_split = np.dot(delta4, self.weights['W4'].T)

        # Separate the deltas for layers 1, 2, and 3
        delta3 = delta4_split[:, -self.hidden3_size:] * self.X_3_activation_derivative(self.a3)

        delta2 = delta4_split[:, -self.hidden3_size-self.hidden2_size:-self.hidden3_size] * self.X_2_activation_derivative(self.a2)
        delta1 = delta4_split[:, :-self.hidden3_size-self.hidden2_size] * self.X_activation_derivative(self.a1)

        # Calculate gradients for layers 1, 2, and 3
        dW3 = np.dot(X.T, delta3)
        db3 = np.sum(delta3, axis=0, keepdims=True)
        dW2 = np.dot(X.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # Update weights and biases
        self.weights['W4'] -= learning_rate * dW4
        self.biases['b4'] -= learning_rate * db4
        self.weights['W3'] -= learning_rate * dW3
        self.biases['b3'] -= learning_rate * db3
        self.weights['W2'] -= learning_rate * dW2
        self.biases['b2'] -= learning_rate * db2
        self.weights['W1'] -= learning_rate * dW1
        self.biases['b1'] -= learning_rate * db1

        return dW4, db4, dW3, db3, dW2, db2, dW1, db1

    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            # Forward pass
            self.feedforward(X)

            # Backpropagation
            self.backpropagation(X, y, learning_rate)

            if epoch % 500 == 0:
                loss = np.mean((self.output - y) ** 2)
                print(f'Epoch {epoch}/{epochs}, Loss: {loss:.4f}')

##########################################################
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import time

iris = load_iris()
X = iris.data
y = iris.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add a small epsilon value to avoid exactly zero or one values
epsilon = 1e-4
X = np.clip(X, epsilon, 1 - epsilon)  # Clip values to be in the range (epsilon, 1 - epsilon)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


learning_rate = 0.00001
epochs = 2500


input_size = 4
hidden1_size = 8
hidden2_size = 8
hidden3_size = 8
output_size = 3
# Create the neural network
nn = NeuralNetwork(input_size, hidden1_size, hidden2_size,hidden3_size, output_size)

# Train the neural network
start_time = time.time()

nn.train(X_train, y_train, learning_rate, epochs)

execution_time = time.time() - start_time

print(f"Training Execution Time: {execution_time:.2f} seconds")
# Evaluate the trained model
def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

y_pred = nn.feedforward(X_test)
acc = accuracy(y_test, y_pred)
print(f'Test Accuracy: {acc * 100:.2f}%')

Epoch 0/2500, Loss: 18.2055
Epoch 500/2500, Loss: 0.2031
Epoch 1000/2500, Loss: 0.1480
Epoch 1500/2500, Loss: 0.1309
Epoch 2000/2500, Loss: 0.1215
Training Execution Time: 0.58 seconds
Test Accuracy: 90.00%


In [50]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time

# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Preprocess the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_onehot = tf.keras.utils.to_categorical(y_encoded, num_classes=3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_onehot, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential()
model.add(Dense(16, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Measure execution time
start_time = time.time()

# Train the model
model.fit(X_train, y_train, epochs=2, batch_size=1, verbose=0)

# Calculate execution time
execution_time = time.time() - start_time

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")
print(f"Execution Time: {execution_time:.2f} seconds")


1/1 [==============================] - 0s 147ms/step - loss: 0.4938 - accuracy: 0.9000
Test Loss: 0.4938, Test Accuracy: 0.9000
Execution Time: 1.37 seconds
